In [1]:
import pandas as pd
import math
import re

In [2]:
pd.options.display.min_rows = 6
pd.options.display.max_rows = 60
pd.options.display.precision = 2

In [3]:
# load stats about longevity per year
df = pd.read_excel('data/Life expectancy in Turkey.xls', skiprows=4, skipfooter=4) \
       .dropna(axis='columns', how='all')

df

,İller-Provinces,Toplam\nTotal,Erkek\nMale,Kadın\nFemale,Toplam\nTotal.1,Erkek\nMale.1,Kadın\nFemale.1,Toplam\nTotal.2,Erkek\nMale.2,Kadın\nFemale.2,Toplam\nTotal.3,Erkek\nMale.3,Kadın\nFemale.3
0,Türkiye-Türkiye,78.00,75.30,80.70,78.00,75.30,80.70,78.00,75.30,80.80,78.33,75.60,81.12
1,Adana,77.13,74.27,80.00,77.39,74.54,80.24,77.53,74.70,80.38,77.66,74.91,80.47
2,Adıyaman,79.57,76.25,82.91,79.55,76.57,82.49,79.71,77.27,82.12,79.29,76.94,81.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Kilis,74.93,71.87,78.00,74.95,72.26,77.64,76.14,72.87,79.55,76.09,73.47,78.83
80,Osmaniye,78.37,75.45,81.35,78.58,75.91,81.25,78.01,75.29,80.81,78.47,76.08,80.91
81,Düzce,78.07,75.29,80.93,77.75,75.01,80.57,77.49,74.49,80.69,78.04,75.19,81.06


In [4]:
df.columns = ['',
              '2013_total', '2013_male', '2013_female',
              '2013-14_total', '2013-14_male', '2013-14_female',
              '2015-17_total', '2015-17_male', '2015-17_female',
              '2018-20_total', '2018-20_male', '2018-20_female']

df = df.set_index('') \
       .rename(index={'Türkiye-Türkiye': 'Türkiye'})

df              

,2013_total,2013_male,2013_female,2013-14_total,2013-14_male,2013-14_female,2015-17_total,2015-17_male,2015-17_female,2018-20_total,2018-20_male,2018-20_female
,,,,,,,,,,,,
Türkiye,78.00,75.30,80.70,78.00,75.30,80.70,78.00,75.30,80.80,78.33,75.60,81.12
Adana,77.13,74.27,80.00,77.39,74.54,80.24,77.53,74.70,80.38,77.66,74.91,80.47
Adıyaman,79.57,76.25,82.91,79.55,76.57,82.49,79.71,77.27,82.12,79.29,76.94,81.67
...,...,...,...,...,...,...,...,...,...,...,...,...
Kilis,74.93,71.87,78.00,74.95,72.26,77.64,76.14,72.87,79.55,76.09,73.47,78.83
Osmaniye,78.37,75.45,81.35,78.58,75.91,81.25,78.01,75.29,80.81,78.47,76.08,80.91
Düzce,78.07,75.29,80.93,77.75,75.01,80.57,77.49,74.49,80.69,78.04,75.19,81.06


In [5]:
# df.drop(columns=['2013_total', '2013_male', '2013_female'], inplace=True)
# df

In [6]:
df = df.loc[:, ['2018-20_total', '2018-20_male', '2018-20_female', '2015-17_total', '2015-17_male', '2015-17_female']]
df.columns = ['overall', 'male', 'female', 'past_overall', 'past_male', 'past_female']
df

,overall,male,female,past_overall,past_male,past_female
,,,,,,
Türkiye,78.33,75.60,81.12,78.00,75.30,80.80
Adana,77.66,74.91,80.47,77.53,74.70,80.38
Adıyaman,79.29,76.94,81.67,79.71,77.27,82.12
...,...,...,...,...,...,...
Kilis,76.09,73.47,78.83,76.14,72.87,79.55
Osmaniye,78.47,76.08,80.91,78.01,75.29,80.81
Düzce,78.04,75.19,81.06,77.49,74.49,80.69


In [7]:
# indexes = df.index.to_list()

# print("dd_ils = {")
# for il in indexes:
#     print(f"    {il:10}: {{'tr_page': '', 'ru': '', 'ru_wiki': '', 'en': '', 'en_wiki': ''}},")
# print("}")

In [8]:
# # create code for placing info in Wikipedia
# def create_table(df, file_header, file_footer, dd_replacement, lang='ru'):

#     def if_value(x, prec=2):
#         return '—' if math.isnan(x) else \
#                f"{x:0.{prec}f}"  if x>=0 else \
#                f"−{-x:0.{prec}f}"                #"{x:0.{prec}f}".format(x, prec)
    
#     def chval(x, prec=2):  # change_value
#         return 'style="background:#fffae0;"|—' if math.isnan(x) else \
#                f'style="background:#fffae0;color:darkgreen;"|{x:0.{prec}f}' if x>0 else \
#                f'style="background:#fffae0;color:crimson;"|−{-x:0.{prec}f}' if x<0 else \
#                f'style="background:#fffae0;color:darkgray;"|−{x:0.{prec}f}'  # normal sorting is required '−' before 0.0 - bug in wiki
    
#     def chval_bold(x, prec=2):  # change_value
#         return '—' if math.isnan(x) else \
#                f'style="background:#fffae0;color:darkgreen;"|\'\'\'{x:0.{prec}f}\'\'\'' if x>0 else \
#                f'style="background:#fffae0;color:crimson;"|\'\'\'−{-x:0.{prec}f}\'\'\'' if x<0 else \
#                f'style="background:#fffae0;color:darkgray;"|\'\'\'{x:0.{prec}f}\'\'\''
    
#     with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
#         table_header = fh.read()
    
#     with open('design/' + file_footer, mode='r', encoding="utf-8") as fh:
#         table_footer = fh.read()

#     st = ''
#     for i in range(len(df)):
#         ser = df.iloc[i]
#         if ser.name == 'Россия в среднем':
#             st += '\n' + '|-class=static-row-header\n' + \
#                   f'|style="text-align:left;"|\'\'\'{dd_replacement[ser.name]}\'\'\' ' + \
#                   f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
#                   f'|| style="text-align:center;background:#eaf3ff;"|\'\'\'{if_value(ser["males"], 2)}\'\'\' ' + \
#                   f'|| style="text-align:center;background:#fee7f6;"|\'\'\'{if_value(ser["females"], 2)}\'\'\' ' + \
#                   f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["females_vs_males"], 2)}\'\'\' ' + \
#                   f'|| style="text-align:center;border-left-width:2px;"|\'\'\'{if_value(ser["urban"], 2)}\'\'\' ' + \
#                   f'|| style="text-align:center;"|\'\'\'{if_value(ser["rural"], 2)}\'\'\' ' + \
#                   f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["urban_vs_rural"], 2)}\'\'\' ' + \
#                   f'|| style="text-align:center;border-left-width:2px;"|\'\'\'{if_value(ser["1991"], 1)}\'\'\' ' + \
#                   f'|| {chval_bold(ser["1991→2000"], 1)} ' + \
#                   f'|| style="text-align:center;"|\'\'\'{if_value(ser["2000"], 1)}\'\'\' ' + \
#                   f'|| {chval_bold(ser["2000→2014"], 2)} ' + \
#                   f'|| style="text-align:center;"|\'\'\'{if_value(ser["2014"], 2)}\'\'\' ' + \
#                   f'|| {chval_bold(ser["2014→2019"], 2)} ' + \
#                   f'|| style="text-align:center;"|\'\'\'{if_value(ser["2019"], 2)}\'\'\' ' + \
#                   f'|| {chval_bold(ser["2019→2020"], 2)} ' + \
#                   f'|| style="text-align:center;"|\'\'\'{if_value(ser["2020"], 2)}\'\'\' ' + \
#                   f'|| {chval_bold(ser["2020→2021"], 2)} ' + \
#                   f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["2021"], 2)}\'\'\' ' + \
#                   f'|| style="color:dimgray;border-left-width:2px;"|\'\'\'{int(ser["pop"])}\'\'\''
#         else:        
#             st += '\n' + '|-\n' + \
#                   f'|style="text-align:left;"|{dd_replacement[ser.name]} ' + \
#                   f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
#                   f'|| style="text-align:center;background:#eaf3ff;"|{if_value(ser["males"], 2)} ' + \
#                   f'|| style="text-align:center;background:#fee7f6;"|{if_value(ser["females"], 2)} ' + \
#                   f'|| style="background:#fff8dc;"|{if_value(ser["females_vs_males"], 2)} ' + \
#                   f'|| style="text-align:center;border-left-width:2px;"|{if_value(ser["urban"], 2)} ' + \
#                   f'|| style="text-align:center;"|{if_value(ser["rural"], 2)} ' + \
#                   f'|| style="background:#fff8dc;"|{if_value(ser["urban_vs_rural"], 2)} ' + \
#                   f'|| style="text-align:center;border-left-width:2px;"|{if_value(ser["1991"], 1)} ' + \
#                   f'|| {chval(ser["1991→2000"], 1)} ' + \
#                   f'|| style="text-align:center;"|{if_value(ser["2000"], 1)} ' + \
#                   f'|| {chval(ser["2000→2014"], 2)} ' + \
#                   f'|| style="text-align:center;"|{if_value(ser["2014"], 2)} ' + \
#                   f'|| {chval(ser["2014→2019"], 2)} ' + \
#                   f'|| style="text-align:center;"|{if_value(ser["2019"], 2)} ' + \
#                   f'|| {chval(ser["2019→2020"], 2)} ' + \
#                   f'|| style="text-align:center;"|{if_value(ser["2020"], 2)} ' + \
#                   f'|| {chval(ser["2020→2021"], 2)} ' + \
#                   f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["2021"], 2)}\'\'\' ' + \
#                   f'|| style="color:dimgray;border-left-width:2px;"|{int(ser["pop"])}'
            
#     if lang == 'ru':
#         st = re.sub('(?<=\d)\.(?=\d)', ',', st)  # replace . to comma, if this . is between two digits

#     st = table_header + st + '\n' + table_footer
    
#     # gray color for missing values
#     st = st.replace(';"|—', ';color:silver;"|—')

#     return st

# table_code = create_table(df_subjects, file_header='rosstat_header_ru.txt', file_footer='rosstat_footer_ru.txt',
#                           dd_replacement=dd_replacement_ru, lang='ru')

In [9]:
df.insert(loc=3, column='f-m', value=df['female']-df['male'])
df.insert(loc=4, column='Δ_years', value=df['overall']-df['past_overall'])
df.insert(loc=8, column='past_f-m', value=df['past_female']-df['past_male'])
df

,overall,male,female,f-m,Δ_years,past_overall,past_male,past_female,past_f-m
,,,,,,,,,
Türkiye,78.33,75.60,81.12,5.52,0.33,78.00,75.30,80.80,5.50
Adana,77.66,74.91,80.47,5.56,0.13,77.53,74.70,80.38,5.67
Adıyaman,79.29,76.94,81.67,4.73,-0.42,79.71,77.27,82.12,4.85
...,...,...,...,...,...,...,...,...,...
Kilis,76.09,73.47,78.83,5.37,-0.05,76.14,72.87,79.55,6.67
Osmaniye,78.47,76.08,80.91,4.82,0.46,78.01,75.29,80.81,5.53
Düzce,78.04,75.19,81.06,5.87,0.55,77.49,74.49,80.69,6.20


In [10]:
# df = df.loc[:, ['2018-20_total', '2018-20_male', '2018-20_female', '2013-14_total', '2015-17_total', '2018-20_total']]
# df.columns = ['total', 'male', 'female', '2013-14', '2015-17', '2018-20']
# df

# df.insert(loc=3, column='f-m', value=df['female']-df['male'])
# df.insert(loc=5, column='Δ1', value=df['2015-17']-df['2013-14'])
# df.insert(loc=7, column='Δ2', value=df['2018-20']-df['2015-17'])
# df

In [11]:
pd.options.display.max_rows = 100
pd.options.display.precision = 3

def sort_df(df, by='overall', ascending=False):
    df.sort_values(by=by, ascending=ascending, inplace=True)
    indexes = df.index.to_list()
    ls_change_order = ['Türkiye']    # 'İstanbul'
    indexes = list(el for el in indexes if el not in ls_change_order)
    indexes = ls_change_order + indexes
    return df.reindex(indexes)
    
df = sort_df(df, by='overall')

In [12]:
dd_replacement = {
    'Türkiye'   : {'tr': ('Türkiye', ''), 'ru': ('Турция', 'Турция'), 'en': ('Turkey', 'Turkey')},
    'Adana'     : {'tr': ('Adana', ''), 'ru': ('Адана', 'Адана (ил)'), 'en': ('Adana', 'Adana Province')},
    'Adıyaman'  : {'tr': ('Adıyaman', ''), 'ru': ('Адыяман', 'Адыяман (ил)'), 'en': ('Adıyaman', 'Adıyaman Province')},
    'Afyonkarahisar': {'tr': ('Afyonkarahisar', ''), 'ru': ('Афьонкарахисар', 'Афьонкарахисар (ил)'), 'en': ('Afyonkarahisar', 'Afyonkarahisar Province')},
    'Aksaray'   : {'tr': ('Aksaray', ''), 'ru': ('Аксарай', 'Аксарай (ил)'), 'en': ('Aksaray', 'Aksaray Province')},
    'Amasya'    : {'tr': ('Amasya', ''), 'ru': ('Амасья', 'Амасья (ил)'), 'en': ('Amasya', 'Amasya Province')},
    'Ankara'    : {'tr': ('Ankara', ''), 'ru': ('Анкара', 'Анкара (ил)'), 'en': ('Ankara', 'Ankara Province')},
    'Antalya'   : {'tr': ('Antalya', ''), 'ru': ('Анталья', 'Анталья (ил)'), 'en': ('Antalya', 'Antalya Province')},
    'Ardahan'   : {'tr': ('Ardahan', ''), 'ru': ('Ардахан', 'Ардахан (ил)'), 'en': ('Ardahan', 'Ardahan Province')},
    'Artvin'    : {'tr': ('Artvin', ''), 'ru': ('Артвин', 'Артвин (ил)'), 'en': ('Artvin', 'Artvin Province')},
    'Aydın'     : {'tr': ('Aydın', ''), 'ru': ('Айдын', 'Айдын (ил)'), 'en': ('Aydın', 'Aydın Province')},
    'Ağrı'      : {'tr': ('Ağrı', ''), 'ru': ('Агры', 'Агры (ил)'), 'en': ('Ağrı', 'Ağrı Province')},
    'Balıkesir' : {'tr': ('Balıkesir', ''), 'ru': ('Балыкесир', 'Балыкесир (ил)'), 'en': ('Balıkesir', 'Balıkesir Province')},
    'Bartın'    : {'tr': ('Bartın', ''), 'ru': ('Бартын', 'Бартын (ил)'), 'en': ('Bartın', 'Bartın Province')},
    'Batman'    : {'tr': ('Batman', ''), 'ru': ('Батман', 'Батман (ил)'), 'en': ('Batman', 'Batman Province')},
    'Bayburt'   : {'tr': ('Bayburt', ''), 'ru': ('Байбурт', 'Байбурт (ил)'), 'en': ('Bayburt', 'Bayburt Province')},
    'Bilecik'   : {'tr': ('Bilecik', ''), 'ru': ('Биледжик', 'Биледжик (ил)'), 'en': ('Bilecik', 'Bilecik Province')},
    'Bingöl'    : {'tr': ('Bingöl', ''), 'ru': ('Бингёль', 'Бингёль (ил)'), 'en': ('Bingöl', 'Bingöl Province')},
    'Bitlis'    : {'tr': ('Bitlis', ''), 'ru': ('Битлис', 'Битлис (ил)'), 'en': ('Bitlis', 'Bitlis Province')},
    'Bolu'      : {'tr': ('Bolu', ''), 'ru': ('Болу', 'Болу (ил)'), 'en': ('Bolu', 'Bolu Province')},
    'Burdur'    : {'tr': ('Burdur', ''), 'ru': ('Бурдур', 'Бурдур (ил)'), 'en': ('Burdur', 'Burdur Province')},
    'Bursa'     : {'tr': ('Bursa', ''), 'ru': ('Бурса', 'Бурса (ил)'), 'en': ('Bursa', 'Bursa Province')},
    'Denizli'   : {'tr': ('Denizli', ''), 'ru': ('Денизли', 'Денизли (ил)'), 'en': ('Denizli', 'Denizli Province')},
    'Diyarbakır': {'tr': ('Diyarbakır', ''), 'ru': ('Диярбакыр', 'Диярбакыр (ил)'), 'en': ('Diyarbakır', 'Diyarbakır Province')},
    'Düzce'     : {'tr': ('Düzce', ''), 'ru': ('Дюздже', 'Дюздже (ил)'), 'en': ('Düzce', 'Düzce Province')},
    'Edirne'    : {'tr': ('Edirne', ''), 'ru': ('Эдирне', 'Эдирне (ил)'), 'en': ('Edirne', 'Edirne Province')},
    'Elazığ'    : {'tr': ('Elazığ', ''), 'ru': ('Элязыг', 'Элязыг (ил)'), 'en': ('Elazığ', 'Elazığ Province')},
    'Erzincan'  : {'tr': ('Erzincan', ''), 'ru': ('Эрзинджан', 'Эрзинджан (ил)'), 'en': ('Erzincan', 'Erzincan Province')},
    'Erzurum'   : {'tr': ('Erzurum', ''), 'ru': ('Эрзурум', 'Эрзурум (ил)'), 'en': ('Erzurum', 'Erzurum Province')},
    'Eskişehir' : {'tr': ('Eskişehir', ''), 'ru': ('Эскишехир', 'Эскишехир (ил)'), 'en': ('Eskişehir', 'Eskişehir Province')},
    'Gaziantep' : {'tr': ('Gaziantep', ''), 'ru': ('Газиантеп', 'Газиантеп (ил)'), 'en': ('Gaziantep', 'Gaziantep Province')},
    'Giresun'   : {'tr': ('Giresun', ''), 'ru': ('Гиресун', 'Гиресун (ил)'), 'en': ('Giresun', 'Giresun Province')},
    'Gümüşhane' : {'tr': ('Gümüşhane', ''), 'ru': ('Гюмюшхане', 'Гюмюшхане (ил)'), 'en': ('Gümüşhane', 'Gümüşhane Province')},
    'Hakkari'   : {'tr': ('Hakkari', ''), 'ru': ('Хаккяри', 'Хаккяри (ил)'), 'en': ('Hakkâri', 'Hakkâri Province')},
    'Hatay'     : {'tr': ('Hatay', ''), 'ru': ('Хатай', 'Хатай (ил)'), 'en': ('Hatay', 'Hatay Province')},
    'Isparta'   : {'tr': ('Isparta', ''), 'ru': ('Ыспарта', 'Ыспарта (ил)'), 'en': ('Isparta', 'Isparta Province')},
    'Iğdır'     : {'tr': ('Iğdır', ''), 'ru': ('Ыгдыр', 'Ыгдыр (ил)'), 'en': ('Iğdır', 'Iğdır Province')},
    'Kahramanmaraş': {'tr': ('Kahramanmaraş', ''), 'ru': ('Кахраманмараш', 'Кахраманмараш (ил)'), 'en': ('Kahramanmaraş', 'Kahramanmaraş Province')},
    'Karabük'   : {'tr': ('Karabük', ''), 'ru': ('Карабюк', 'Карабюк (ил)'), 'en': ('Karabük', 'Karabük Province')},
    'Karaman'   : {'tr': ('Karaman', ''), 'ru': ('Караман', 'Караман (ил)'), 'en': ('Karaman', 'Karaman Province')},
    'Kars'      : {'tr': ('Kars', ''), 'ru': ('Карс', 'Карс (ил)'), 'en': ('Kars', 'Kars Province')},
    'Kastamonu' : {'tr': ('Kastamonu', ''), 'ru': ('Кастамону', 'Кастамону (ил)'), 'en': ('Kastamonu', 'Kastamonu Province')},
    'Kayseri'   : {'tr': ('Kayseri', ''), 'ru': ('Кайсери', 'Кайсери (ил)'), 'en': ('Kayseri', 'Kayseri Province')},
    'Kilis'     : {'tr': ('Kilis', ''), 'ru': ('Килис', 'Килис (ил)'), 'en': ('Kilis', 'Kilis Province')},
    'Kocaeli'   : {'tr': ('Kocaeli', ''), 'ru': ('Коджаэли', 'Коджаэли (ил)'), 'en': ('Kocaeli', 'Kocaeli Province')},
    'Konya'     : {'tr': ('Konya', ''), 'ru': ('Конья', 'Конья (ил)'), 'en': ('Konya', 'Konya Province')},
    'Kütahya'   : {'tr': ('Kütahya', ''), 'ru': ('Кютахья', 'Кютахья (ил)'), 'en': ('Kütahya', 'Kütahya Province')},
    'Kırklareli': {'tr': ('Kırklareli', ''), 'ru': ('Кыркларели', 'Кыркларели (ил)'), 'en': ('Kırklareli', 'Kırklareli Province')},
    'Kırıkkale' : {'tr': ('Kırıkkale', ''), 'ru': ('Кырыккале', 'Кырыккале (ил)'), 'en': ('Kırıkkale', 'Kırıkkale Province')},
    'Kırşehir'  : {'tr': ('Kırşehir', ''), 'ru': ('Кыршехир', 'Кыршехир (ил)'), 'en': ('Kırşehir', 'Kırşehir Province')},
    'Malatya'   : {'tr': ('Malatya', ''), 'ru': ('Малатья', 'Малатья (ил)'), 'en': ('Malatya', 'Malatya Province')},
    'Manisa'    : {'tr': ('Manisa', ''), 'ru': ('Маниса', 'Маниса (ил)'), 'en': ('Manisa', 'Manisa Province')},
    'Mardin'    : {'tr': ('Mardin', ''), 'ru': ('Мардин', 'Мардин (ил)'), 'en': ('Mardin', 'Mardin Province')},
    'Mersin'    : {'tr': ('Mersin', ''), 'ru': ('Мерсин', 'Мерсин (ил)'), 'en': ('Mersin', 'Mersin Province')},
    'Muğla'     : {'tr': ('Muğla', ''), 'ru': ('Мугла', 'Мугла (ил)'), 'en': ('Muğla', 'Muğla Province')},
    'Muş'       : {'tr': ('Muş', ''), 'ru': ('Муш', 'Муш (ил)'), 'en': ('Muş', 'Muş Province')},
    'Nevşehir'  : {'tr': ('Nevşehir', ''), 'ru': ('Невшехир', 'Невшехир (ил)'), 'en': ('Nevşehir', 'Nevşehir Province')},
    'Niğde'     : {'tr': ('Niğde', ''), 'ru': ('Нигде', 'Нигде (ил)'), 'en': ('Niğde', 'Niğde Province')},
    'Ordu'      : {'tr': ('Ordu', ''), 'ru': ('Орду', 'Орду (ил)'), 'en': ('Ordu', 'Ordu Province')},
    'Osmaniye'  : {'tr': ('Osmaniye', ''), 'ru': ('Османие', 'Османие (ил)'), 'en': ('Osmaniye', 'Osmaniye Province')},
    'Rize'      : {'tr': ('Rize', ''), 'ru': ('Ризе', 'Ризе (ил)'), 'en': ('Rize', 'Rize Province')},
    'Sakarya'   : {'tr': ('Sakarya', ''), 'ru': ('Сакарья', 'Сакарья (ил)'), 'en': ('Sakarya', 'Sakarya Province')},
    'Samsun'    : {'tr': ('Samsun', ''), 'ru': ('Самсун', 'Самсун (ил)'), 'en': ('Samsun', 'Samsun Province')},
    'Siirt'     : {'tr': ('Siirt', ''), 'ru': ('Сиирт', 'Сиирт (ил)'), 'en': ('Siirt', 'Siirt Province')},
    'Sinop'     : {'tr': ('Sinop', ''), 'ru': ('Синоп', 'Синоп (ил)'), 'en': ('Sinop', 'Sinop Province')},
    'Sivas'     : {'tr': ('Sivas', ''), 'ru': ('Сивас', 'Сивас (ил)'), 'en': ('Sivas', 'Sivas Province')},
    'Tekirdağ'  : {'tr': ('Tekirdağ', ''), 'ru': ('Текирдаг', 'Текирдаг (ил)'), 'en': ('Tekirdağ', 'Tekirdağ Province')},
    'Tokat'     : {'tr': ('Tokat', ''), 'ru': ('Токат', 'Токат (ил)'), 'en': ('Tokat', 'Tokat Province')},
    'Trabzon'   : {'tr': ('Trabzon', ''), 'ru': ('Трабзон', 'Трабзон (ил)'), 'en': ('Trabzon', 'Trabzon Province')},
    'Tunceli'   : {'tr': ('Tunceli', ''), 'ru': ('Тунджели', 'Тунджели (ил)'), 'en': ('Tunceli', 'Tunceli Province')},
    'Uşak'      : {'tr': ('Uşak', ''), 'ru': ('Ушак', 'Ушак (ил)'), 'en': ('Uşak', 'Uşak Province')},
    'Van'       : {'tr': ('Van', ''), 'ru': ('Ван', 'Ван (ил)'), 'en': ('Van', 'Van Province')},
    'Yalova'    : {'tr': ('Yalova', ''), 'ru': ('Ялова', 'Ялова (ил)'), 'en': ('Yalova', 'Yalova Province')},
    'Yozgat'    : {'tr': ('Yozgat', ''), 'ru': ('Йозгат', 'Йозгат (ил)'), 'en': ('Yozgat', 'Yozgat Province')},
    'Zonguldak' : {'tr': ('Zonguldak', ''), 'ru': ('Зонгулдак', 'Зонгулдак (ил)'), 'en': ('Zonguldak', 'Zonguldak Province')},
    'Çanakkale' : {'tr': ('Çanakkale', ''), 'ru': ('Чанаккале', 'Чанаккале (ил)'), 'en': ('Çanakkale', 'Çanakkale Province')},
    'Çankırı'   : {'tr': ('Çankırı', ''), 'ru': ('Чанкыры', 'Чанкыры (ил)'), 'en': ('Çankırı', 'Çankırı Province')},
    'Çorum'     : {'tr': ('Çorum', ''), 'ru': ('Чорум', 'Чорум (ил)'), 'en': ('Çorum', 'Çorum Province')},
    'İstanbul'  : {'tr': ('İstanbul', ''), 'ru': ('Стамбул', 'Стамбул (ил)'), 'en': ('Istanbul', 'Istanbul Province')},
    'İzmir'     : {'tr': ('İzmir', ''), 'ru': ('Измир', 'Измир (ил)'), 'en': ('İzmir', 'İzmir Province')},
    'Şanlıurfa' : {'tr': ('Şanlıurfa', ''), 'ru': ('Шанлыурфа', 'Шанлыурфа (ил)'), 'en': ('Şanlıurfa', 'Şanlıurfa Province')},
    'Şırnak'    : {'tr': ('Şırnak', ''), 'ru': ('Ширнак', 'Ширнак (ил)'), 'en': ('Şırnak', 'Şırnak Province')},
}

In [13]:
# indexes = df.index.to_list()

# print("dd_ils = {")
# for il in indexes:
#     il_quoted = "'" + il + "'"
#     print(f"    {il_quoted:12}: {{'tr': ('{il}', ''), 'ru': ('{dd_replacement[il]['ru']}', '{dd_replacement[il]['ru_wiki']}'), 'en': ('{dd_replacement[il]['en']}', '{dd_replacement[il]['en_wiki']}')}},")
# print("}")

In [14]:
# ser = 'İstanbul'
# lang = 'ru'
# lang_wiki = 'ru_wiki'

# dd_replacement[ser][lang], dd_replacement[ser][lang_wiki]

https://ru.wikipedia.org/wiki/Административное_деление_Турции<br>
https://en.wikipedia.org/wiki/Provinces_of_Turkey<br>
https://tr.wikipedia.org/wiki/Türkiye'nin_ilçeleri

In [15]:
# create code for placing info in Wikipedia
def create_table(df, file_header, file_footer, lang='ru'):

    def if_value(x, prec=2):
        return '—' if math.isnan(x) else \
               f"{x:0.{prec}f}"  if x>=0 else \
               f"−{-x:0.{prec}f}"                #"{x:0.{prec}f}".format(x, prec)
    
    def chval(x, prec=2):  # change_value
        return 'style="padding-right:1em;"|—' if math.isnan(x) else \
               f'style="padding-right:1em;color:darkgreen;"|{x:0.{prec}f}' if x>0 else \
               f'style="padding-right:1em;color:crimson;"|−{-x:0.{prec}f}' if x<0 else \
               f'style="padding-right:1em;color:darkgray;"|−{x:0.{prec}f}'  # normal sorting is required '−' before 0.0 - bug in wiki

    def chval_bold(x, prec=2):  # change_value
        return '—' if math.isnan(x) else \
               f'style="padding-right:1em;color:darkgreen;"|\'\'\'{x:0.{prec}f}\'\'\'' if x>0 else \
               f'style="padding-right:1em;color:crimson;"|\'\'\'−{-x:0.{prec}f}\'\'\'' if x<0 else \
               f'style="padding-right:1em;color:darkgray;"|\'\'\'{x:0.{prec}f}\'\'\''                 #  background:#fffae0;
    
    with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
        table_header = fh.read()
    
    with open('design/' + file_footer, mode='r', encoding="utf-8") as fh:
        table_footer = fh.read()

    st = ''
    for i in range(len(df)):
        ser = df.iloc[i]
        if ser.name == 'Türkiye':
             st += '\n' + '|-class=static-row-header\n' + \
                  f'|style="text-align:left;"|\'\'\'{dd_replacement[ser.name][lang][0]}\'\'\' ' + \
                    f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;background:#eaf3ff;"|\'\'\'{if_value(ser["male"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;background:#fee7f6;"|\'\'\'{if_value(ser["female"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;background:#fff8dc;"|\'\'\'{if_value(ser["f-m"], 2)}\'\'\' ' + \
                  f'|| {chval_bold(ser["Δ_years"], 2)} ' + \
                  f'|| style="text-align:center;background:#e0ffd8;border-left-width:2px;"|\'\'\'{if_value(ser["past_overall"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;background:#eaf3ff;"|\'\'\'{if_value(ser["past_male"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;background:#fee7f6;"|\'\'\'{if_value(ser["past_female"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;background:#fff8dc;"|\'\'\'{if_value(ser["f-m"], 2)}\'\'\' '
        else:        
            st += '\n' + '|-\n' + \
                  f'|style="text-align:left;"|[[{dd_replacement[ser.name][lang][1]}|{dd_replacement[ser.name][lang][0]}]] ' + \
                  f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;background:#eaf3ff;"|{if_value(ser["male"], 2)} ' + \
                  f'|| style="text-align:center;background:#fee7f6;"|{if_value(ser["female"], 2)} ' + \
                  f'|| style="text-align:center;background:#fff8dc;"|{if_value(ser["f-m"], 2)} ' + \
                  f'|| {chval(ser["Δ_years"], 2)} ' + \
                  f'|| style="text-align:center;background:#e0ffd8;border-left-width:2px;"|\'\'\'{if_value(ser["past_overall"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;background:#eaf3ff;"|{if_value(ser["past_male"], 2)} ' + \
                  f'|| style="text-align:center;background:#fee7f6;"|{if_value(ser["past_female"], 2)} ' + \
                  f'|| style="text-align:center;background:#fff8dc;"|{if_value(ser["f-m"], 2)} '
            
    if lang == 'ru':
        st = re.sub('(?<=\d)\.(?=\d)', ',', st)  # replace . to comma, if this . is between two digits

    st = table_header + st + '\n' + table_footer
    
    # gray color for missing values
    st = st.replace(';"|—', ';color:silver;"|—')

    return st


table_code = create_table(df, file_header='turkstat_header_ru.txt', file_footer='turkstat_footer_ru.txt', lang='ru')

# # write the code to file
# with open('output/Table code for Turkish regions -ru.txt', 'w', encoding="utf-8") as fh:
#     fh.write(table_code)

In [16]:
table_code = create_table(df, file_header='turkstat_header_en.txt', file_footer='turkstat_footer_en.txt', lang='en')

# # write the code to file
# with open('output/Table code for Turkish regions -en.txt', 'w', encoding="utf-8") as fh:
#     fh.write(table_code)

<br>
<br>
<br>
<hr>

In [17]:
df.head()

,overall,male,female,f-m,Δ_years,past_overall,past_male,past_female,past_f-m
,,,,,,,,,
Türkiye,78.334,75.604,81.124,5.519,0.334,78.000,75.300,80.800,5.500
Gümüşhane,80.781,78.074,83.544,5.470,1.007,79.774,76.254,83.456,7.202
Muğla,80.702,77.748,83.964,6.217,0.424,80.278,77.591,83.179,5.587
Tunceli,80.331,77.015,84.060,7.045,-0.359,80.690,77.409,84.248,6.838
Trabzon,80.287,76.772,83.788,7.016,0.290,79.997,76.480,83.375,6.895


In [18]:
df_coloring = df.loc[:, ['overall', 'past_overall']] \
                .round(2)
df_coloring.head()

,overall,past_overall
,,
Türkiye,78.33,78.00
Gümüşhane,80.78,79.77
Muğla,80.70,80.28
Tunceli,80.33,80.69
Trabzon,80.29,80.00


In [19]:
df_coloring.sort_index(inplace=True)
indexes = df_coloring.index.to_list()
ls_change_order = ['Türkiye']    # 'İstanbul'
indexes = list(el for el in indexes if el not in ls_change_order)
indexes = ls_change_order + indexes
df_coloring = df_coloring.reindex(indexes)
df_coloring.head()

,overall,past_overall
,,
Türkiye,78.33,78.00
Adana,77.66,77.53
Adıyaman,79.29,79.71
Afyonkarahisar,76.96,77.07
Aksaray,78.51,78.32


In [20]:
define_color = lambda v: ('#005203' if v >= 81 else
                          '#3faf46' if v >= 80 else
                          '#81d41a' if v >= 79 else
                          '#ffd428' if v >= 78 else
                          '#ff972f' if v >= 77 else
                          '#ff0000' if v >= 76 else
                          '#c40000')

In [21]:
df_coloring.insert(loc=1, column='color_1', value=df['overall'].map(define_color))
df_coloring.insert(loc=3, column='color_2', value=df['past_overall'].map(define_color))
df_coloring.head()

,overall,color_1,past_overall,color_2
,,,,
Türkiye,78.33,#ffd428,78.00,#ffd428
Adana,77.66,#ff972f,77.53,#ff972f
Adıyaman,79.29,#81d41a,79.71,#81d41a
Afyonkarahisar,76.96,#ff0000,77.07,#ff972f
Aksaray,78.51,#ffd428,78.32,#ffd428


In [23]:
df_coloring.to_csv('output/coloring of Turkish provinces.csv')